In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('https://github.com/Azidalus/Business-metrics-analysis-in-Pandas/blob/main/entries.csv?raw=true')
df

,user,dt
0,0,12.11.2023
1,0,13.11.2023
2,0,14.11.2023
3,0,16.11.2023
4,0,17.11.2023
...,...,...
52320,1998,26.12.2023
52321,1998,18.06.2024
52322,1999,04.06.2023
52323,1999,14.07.2023


In [3]:
# Convert to date format
df['dt'] = pd.to_datetime(df['dt'], format='%d.%m.%Y')
df.head()

,user,dt
0,0,2023-11-12
1,0,2023-11-13
2,0,2023-11-14
3,0,2023-11-16
4,0,2023-11-17


## Q1 - Payback

### Q1.1: Does the total payment for all first subscribtions cover the total cost of acquisition?

Let's break down the task: 
1. Calculate total payment for all first subscriptions
    - `Number of payments for first subscription` * `Subscription cost`
2. Calculate total cost of acquisition
    - `Number of users` * `Acquisition cost`
3. Look at the difference between 1. and 2.

#### 1. Calculate total payment for first subscription

In [5]:
# Create column with registrtation date
df['reg'] = df.groupby('user')['dt'].transform('min')
df

,user,dt,reg
0,0,2023-11-12,2023-11-12
1,0,2023-11-13,2023-11-12
2,0,2023-11-14,2023-11-12
3,0,2023-11-16,2023-11-12
4,0,2023-11-17,2023-11-12
...,...,...,...
52320,1998,2023-12-26,2023-07-26
52321,1998,2024-06-18,2023-07-26
52322,1999,2023-06-04,2023-06-04
52323,1999,2023-07-14,2023-06-04


In [6]:
# Create a column storing the difference between the registration date and interaction dates
df['diff'] = (df['dt'] - df['reg']).dt.days
df

,user,dt,reg,diff
0,0,2023-11-12,2023-11-12,0
1,0,2023-11-13,2023-11-12,1
2,0,2023-11-14,2023-11-12,2
3,0,2023-11-16,2023-11-12,4
4,0,2023-11-17,2023-11-12,5
...,...,...,...,...
52320,1998,2023-12-26,2023-07-26,153
52321,1998,2024-06-18,2023-07-26,328
52322,1999,2023-06-04,2023-06-04,0
52323,1999,2023-07-14,2023-06-04,40


In [57]:
Revenue = df[df['diff'] > 14]['user'].nunique() * 9.9
Revenue

19611.9

#### 2. Calculate total cost of acquisition

In [63]:
Acquisition = df['user'].nunique() * 5
Acquisition

10000

#### 3. Answer: Revenue covers Acquisition

### Q1.2 On what day do we start to cover acquisition costs?

In [64]:
from math import ceil

# We need the following number of users to cover acquisition costs:
ceil(df['user'].nunique() * 5 / 9.9)

1011

In [62]:
(
    df[df['diff'] >= 14]                         # Look only at rows after showing the paywall
    .groupby('user')                             # Make a df consisting of a user and their minimal date they showed up in df
    .agg(min_diff=('diff', 'min'))
    .reset_index()
    .sort_values('min_diff')                     # Sort it by that minimal date 
    .head(ceil(df['user'].nunique() * 5 / 9.9))  # Take exactly the number of users that is needed to cover acquisition costs
    .tail(1)                                     # Look at the minimal date field in the last record
    ['min_diff']
)

1836    16
Name: min_diff, dtype: int64

## Q2 - Percentage of buyers

### Q2.1 What percentage of users buy the subscription immediately on day 14?

In [53]:
print(df[df['diff'] == 14]['user'].nunique() / df['user'].nunique() * 100, '%')

25.0 %


### Q2.2 What percentage of users buy the subscription later?

In [22]:
print((df[df['diff'] > 14]['user'].nunique() - df[df['diff'] == 14]['user'].nunique()) / df['user'].nunique()*100, '%')

74.05000000000001 %


## Q3 - n-day retention

### Q3.1 By how many percentage points does n-day retention drop after showing the paywall?

In [24]:
df.head()

,user,dt,reg,diff
0,0,2023-11-12,2023-11-12,0
1,0,2023-11-13,2023-11-12,1
2,0,2023-11-14,2023-11-12,2
3,0,2023-11-16,2023-11-12,4
4,0,2023-11-17,2023-11-12,5


#### 1. Compute n-day retention

In [13]:
retention_df = (
    df
    .groupby('diff')
    .agg(unique_users=('user','count'))
    .reset_index()
)

retention_df['retention'] = retention_df['unique_users'] / retention_df.loc[0,'unique_users'] * 100
retention_df

,diff,unique_users,retention
0,0,2000,100.00
1,1,1039,51.95
2,2,1052,52.60
3,3,1055,52.75
4,4,1024,51.20
...,...,...,...
359,360,6,0.30
360,361,5,0.25
361,362,3,0.15
362,363,9,0.45


#### 2. Calculate the difference in retention

In [14]:
retention_df.loc[13:14,'retention']

13    43.0
14    25.0
Name: retention, dtype: float64

In [26]:
print(retention_df.loc[13,'retention'] - retention_df.loc[14,'retention'], '%')

18.0 %


### Q3.2 On what day is the peak value of n-day retention observed after showing the paywall?

In [39]:
retention_df[14:].iloc[retention_df[14:]['retention'].argmax()]

diff             16.0
unique_users    518.0
retention        25.9
Name: 16, dtype: float64